In [ ]:
import pandas as pd  
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

## Task 1

In [ ]:
# Load the data
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
boston = pd.read_csv('housing.data', header=None, delim_whitespace = True, names=column_names)

In [ ]:
# Check the first 5 rows of the data
boston.head()

In [ ]:
# Check the shape of the data
boston.shape

In [ ]:
# displaying the info of the dataframe for basic checks on the number of data entries, data type, etc
boston.info()

In [ ]:
# verification of missing data
boston.isnull().sum()

In [ ]:
# displaying the basic statistics of the dataframe
boston.describe()

In [ ]:
# plotting the distribution of the output variable
sns.distplot(boston['MEDV'])
plt.show()

# the dependent variable follows a normal distribution

## Task 2

## Task 3

In [ ]:
# Putting feature variable to X
x = boston[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']]

# Putting response variable to y
y = boston['MEDV']

In [ ]:
# To check p-value of variables
import statsmodels.api as sm          # Importing statsmodels
x = sm.add_constant(x)    # Adding a constant column to our dataframe
# create a first fitted model
lm = sm.OLS(y,x).fit()
# Summary of linear model
print(lm.summary())

In [ ]:
# UDF for calculating vif value
def vif_cal(input_data, dependent_col):
    vif_df = pd.DataFrame( columns = ['Var', 'Vif'])
    x_vars=input_data.drop([dependent_col], axis=1)
    xvar_names=x_vars.columns
    for i in range(0,xvar_names.shape[0]):
        y=x_vars[xvar_names[i]] 
        x=x_vars[xvar_names.drop(xvar_names[i])]
        rsq=sm.OLS(y,x).fit().rsquared 
        vif=round(1/(1-rsq),2)
        vif_df.loc[i] = [xvar_names[i], vif]
    return vif_df.sort_values(by = 'Vif', axis=0, ascending=False, inplace=False)

In [ ]:
# Calculating Vif value
vif_cal(input_data=boston, dependent_col="MEDV")

In [ ]:
# Plotting input variables 'B', 'DIS', 'LSTAT', 'ZN', 'CRIM' against the output variable 'MEDV'
boston_5 = boston[['B', 'DIS', 'LSTAT', 'ZN', 'CRIM', 'MEDV']]

In [ ]:
# Print the correlation matrix
plt.figure(figsize=(15, 10))
correlation_matrix = boston_5.corr()
sns.heatmap(data=correlation_matrix, annot=True, annot_kws={"size": 20})

In [ ]:
sns.pairplot(boston_5)

In [ ]:
# plotting LSTAT vs output variable MEDV to show example of correlated relationship
plt.figure(figsize=(15, 10))
plt.scatter(boston['MEDV'], boston['LSTAT'])

plt.title("Plot of LSTAT VS MEDV")
plt.xlabel('MEDV')
plt.ylabel('LSTAT')

plt.show()

In [ ]:
# plotting output variable MEDV vs DIS to show example of not highly correlated relationship
plt.figure(figsize=(15, 10))
plt.scatter(boston['MEDV'], boston['DIS'])

plt.title("Plot of DIS VS MEDV")
plt.xlabel('MEDV')
plt.ylabel('DIS')

plt.show()

In [ ]:
# plotting output variable MEDV vs DIS to show example of not highly correlated relationship
plt.figure(figsize=(15, 10))
plt.scatter(boston['MEDV'], boston['ZN'])

plt.title("Plot of ZN VS MEDV")
plt.xlabel('MEDV')
plt.ylabel('ZN')

plt.show()

## Task 4

In [ ]:
# RMSE funtion = square root of MSE
def RMSE(y, y_pred):
    MSE = np.sum((y_pred - y)**2)/y.shape[0]
    RMSE = MSE**0.5
    
    return RMSE

In [ ]:
# R2 funtion = (1 - RSS/ TSS)
def r2(y, y_pred):
    RSS = np.sum(((y - y_pred)**2))
    TSS = np.sum(((y - y.mean())**2))
    r2 = 1-(RSS/TSS)
    
    return r2

## Task 5

In [ ]:
def  cal_cost(theta,X,y):
    '''
    theta = coeff vector
    X     = input dataset (13 attributes)
    y     = output (1 variable)
    '''
    m = len(y)
    
    prediction = np.dot(X,theta)
    cost = 1/(2*m) * np.sum((prediction-y)**2)
    
    return cost

In [ ]:
def gradient_descent(X,y,theta,learning_rate=0.01,iterations=100):
    '''
    X     = input dataset (13 attributes)
    y     = output (1 variable)
    theta = coeff vector
    learning_rate = default value of 0.01
    iterations = no of iterations
    
    Returns the final theta vector and array of cost history over no of iterations
    '''
    m = len(y)
    
    cost_history = np.zeros(iterations)
    theta_history = np.zeros((iterations, len(theta)))
    iter_counter = 0
    threshold = 0.001 # threshold declaring convergence, i.e. change in loss < 0.001
    
    for it in range(iterations):
        prediction = np.dot(X,theta) # y_pred for each X data pt
        d_theta = (1/m)*(np.dot(X.T,(prediction - y))) # gradient of theta
        theta = theta - learning_rate*d_theta # new theta after first iteration
        
        theta_history[it,:] = theta.T
        cost_history[it]  = cal_cost(theta,X,y)
        
        iter_counter += 1
        
        # uncomment the next 4 lines of code to run iterations until convergence threshold is reached
#         if it > 0 and (cost_history[it-1]-cost_history[it])<threshold:
#             break
            
#     print("No. of iterations ran:", iter_counter)
        
    return theta, cost_history, theta_history

## Task 6

In [ ]:
# Normalize the 13 input features using (data-min)/(max-min)
def dataNorm(X):
    # Normalize the dataset
    X_norm = np.array(X)    # initializes array X_norm with the same values as array x
    for i in range(X.shape[1]-1):   # updates X_norm with the normalized input attributes
        X_norm[:,i] = (X_norm[:,i]-np.min(X[:,i]))/(np.max(X[:,i])-np.min(X[:,i]))
    return X_norm

In [ ]:
boston = boston.to_numpy()
boston_norm = dataNorm(boston)

## Task 7

In [ ]:
# Split dataset into train (90%) and test (10%)
def splitTT(X_norm, percentTrain):
    # Split the dataset into training and testing using the train-and-test split method
    percentTrain = float(percentTrain)
    if percentTrain > 1 or percentTrain < 0:    #checks that expected portion of dataset for training the domain is (0..1)
        print("Invalid training split size")
    else:
        np.random.shuffle(X_norm)
        X_train = X_norm[:round(percentTrain * X_norm.shape[0]), :]
        X_test = X_norm[round(percentTrain * X_norm.shape[0]):, :]
        X_split = [X_train, X_test]
        return X_split

In [ ]:
X_split = splitTT(boston_norm,0.9)

## Task 8

In [ ]:
# Extracting the X train dataset
X_train_ori = np.array(X_split[0][:,:-1])
const_col = np.ones((len(X_train_ori),1)) # initialize a column of 1s
X_train = np.append(const_col, X_train_ori, axis=1)  # adding a constant to the array

# Extracting the y train dataset
y_train_ori = np.array(X_split[0][:,-1])
y_train = y_train_ori.reshape(-1, 1)

# Extracting the X test dataset
X_test_ori = np.array(X_split[1][:,:-1])
const_col = np.ones((len(X_test_ori),1)) # initialize a column of 1s
X_test = np.append(const_col, X_test_ori, axis=1) # adding a constant to the array

# Extracting the y test dataset
y_test_ori = np.array(X_split[1][:,-1])
y_test = y_test_ori.reshape(-1, 1)

In [ ]:
# Initializing the learning rate and no. of iterations
lr = 0.03
n_iter = 20000

# Initial values of parameters set as 0s
theta = np.zeros((len(X_train[0]),1))

# Running the gradient descent algorithm to compute the parameters
theta,cost_history,theta_history = gradient_descent(X_train, y_train, theta, lr, n_iter)
print('Final Cost (MSE):  {:0.3f}'.format(cost_history[-1]))

# Calculating RMSE and R2 score using train dataset and computed parameters
y_prediction_train = np.dot(X_train,theta)
print("\nTrain RMSE using user constructed linear regression model:",
round(RMSE(y_train, y_prediction_train),3))
print("\nTrain R2 using user constructed linear regression model:",
round(r2(y_train, y_prediction_train),3))

# Calculating RMSE and R2 score using test dataset and computed parameters
y_prediction_test = np.dot(X_test,theta)
print("\nTest RMSE using user constructed linear regression model:",
round(RMSE(y_test, y_prediction_test),3))
print("\nTest R2 using user constructed linear regression model:",
round(r2(y_test, y_prediction_test),3))

In [ ]:
# Plot of Cost vs iterations
plt.figure(figsize=(12, 8))
plt.plot(range(10000), cost_history[:10000,])
plt.xticks(np.arange(0, 10001, 1000))

plt.title("Cost vs iterations")
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.show()

In [ ]:
# If iterations are too low
plt.figure(figsize=(12, 8))
plt.plot(range(250), cost_history[:250], 'b.')

# Min cost line based on experimental run. Value of y has to be changed accordingly run each run
plt.axhline(y = 10.7, color = 'g', linestyle = '--', label = 'Min cost') 

plt.xticks(np.arange(0, 251, 50))
plt.title("Cost vs iterations")
plt.legend()
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.show()

In [ ]:
# Iterations threshold where graph converges
plt.figure(figsize=(12, 8))
plt.plot(range(4000), cost_history[:4000])

# Threshold and Min cost line based on experimental run. Value of x and y has to be changed accordingly for each run
plt.axvline(x = 1994, color = 'g', linestyle = '--', label = 'threshold')
plt.axhline(y = 10.7, color = 'r', linestyle = '--', label = 'Min cost')

plt.xticks(np.arange(0, 4001, 500))
plt.title("Cost vs iterations")
plt.legend()
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.show()

In [ ]:
# Plot of predicted_y_train VS actual_y_train
plt.figure(figsize=(10,10))
plt.scatter(y_train, y_prediction_train)

p1 = min(min(y_train), min(y_prediction_train))
p2 = max(max(y_train), max(y_prediction_train))

plt.title("Plot of predicted_y_train VS actual_y_train")
plt.plot([p1, p2], [p1, p2], 'k--', lw=4)
plt.xlabel('actual_y_train', fontsize=15)
plt.ylabel('predicted_y_train', fontsize=15)
plt.axis('equal')
plt.show()

In [ ]:
# Plot of predicted_y_test VS actual_y_test
plt.figure(figsize=(10,10))
plt.scatter(y_test, y_prediction_test)

p1 = min(min(y_test), min(y_prediction_test))
p2 = max(max(y_test), max(y_prediction_test))

plt.title("Plot of predicted_y_test VS actual_y_test")
plt.plot([p1, p2], [p1, p2], 'k--', lw=4)
plt.xlabel('actual_y_test', fontsize=15)
plt.ylabel('predicted_y_test', fontsize=15)
plt.axis('equal')
plt.show()

In [ ]:
# Comparison of different learning rate
fig = plt.figure(figsize=(15, 10))
n_iter = 500
theta = np.zeros((len(X_train[0]),1))

lr1 = 0.001
theta1,cost_history1,theta_history1 = gradient_descent(X_train, y_train, theta, lr1, n_iter)
plt.plot(range(n_iter), cost_history1, 'b.', label = 'learning rate 0.001')

lr2 = 0.003
theta2,cost_history2,theta_history2 = gradient_descent(X_train, y_train, theta, lr2, n_iter)
plt.plot(range(n_iter), cost_history2, 'y.', label = 'learning rate 0.003')

lr3 = 0.01
theta3,cost_history3,theta_history3 = gradient_descent(X_train, y_train, theta, lr3, n_iter)
plt.plot(range(n_iter), cost_history3, 'm.', label = 'learning rate 0.01')

lr4 = 0.03
theta4,cost_history4,theta_history4 = gradient_descent(X_train, y_train, theta, lr4, n_iter)
plt.plot(range(n_iter), cost_history4, 'k.', label = 'learning rate 0.03')

lr5 = 0.3
theta5,cost_history5,theta_history5 = gradient_descent(X_train, y_train, theta, lr5, n_iter)
plt.plot(range(n_iter), cost_history5, 'g.', label = 'learning rate 0.3')

# Min cost line based on experimental run. Value of y has to be changed accordingly
plt.axhline(y = 10.7, color = 'r', linestyle = '--', label = 'Min cost')

plt.xticks(np.arange(0, n_iter+1, 50))
plt.title("Cost vs iterations")
plt.legend(fontsize = 'x-large', markerscale = 3)
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.show()

In [ ]:
# Learning rate of 0.55
fig = plt.figure(figsize=(15, 10))
n_iter = 30
theta = np.zeros((len(X_train[0]),1))

lr6 = 0.55
theta6,cost_history6,theta_history6 = gradient_descent(X_train, y_train, theta, lr6, n_iter)
plt.plot(range(n_iter), cost_history6, 'b.', label = 'learning rate 0.55', markersize = 25)

# Min cost line based on experimental run. Value of y has to be changed accordingly
plt.axhline(y = 10.7, color = 'r', linestyle = '--', label = 'Min cost')

plt.xticks(np.arange(0, n_iter+1, 1))
plt.title("Cost vs iterations")
plt.legend(fontsize = 'x-large')
plt.xlabel("Iterations")
plt.ylabel("Cost")
plt.show()

## Task 9

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

reg = LinearRegression().fit(X_train, y_train)

y_pred_train = reg.predict(X_train)
print("\nTrain RMSE using sklearn linear regression model:",
round(np.sqrt(mean_squared_error(y_train, y_pred_train)),3))
print("\nTrain R2 using sklearn linear regression model:",
round(r2_score(y_train, y_pred_train),3))

y_pred_test = reg.predict(X_test)
print("\nTest RMSE using sklearn linear regression model:",
round(np.sqrt(mean_squared_error(y_test, y_pred_test)),3))
print("\nTest R2 using sklearn linear regression model:",
round(r2_score(y_test, y_pred_test),3))